# 데이터베이스와 SQL
데이터는 종종 데이터베이스(database)에 저장되어 제공된다. DB는 데이터를 효과적으로 **저장**하고 질의 또는 **쿼리(query)** 를 하기 위한 시스템으로,
이중 대부분은 PostgreSQL, MySQL, SQL Server 등과 같이 데이터를 **테이블** 에 저장하고 데이터를 조작할 수 있는 선언적(declarative) 언어인
SQL(Structured Query Language)로 쿼리하는 **관계형 데이터베이스(relational database)**이다.

SQL은 데이터 과학자에게 필수적인 도구 중 하나이다. 

이 장에서는 NotQuiteABase라는 파이썬 구현체를 만들 것이다. NotQuiteABase에 SQL을 실제로 적용하면서, SQL을 어떤 식으로 활용하는지 살펴볼 예정이다.

## 24.1 CREATE TABLE과 INSERT

관계형 데이터베이스는 여러 테이블의 집합(과 그들 간의 관계)이다. 테이블은 지금까지 우리가 다뤘던 행렬들과 크게 다르지 않은 **행의 집합** 이다.

가장 중요한 **차이점** 은, 테이블에는 **열의 이름** (관계형 데이터베이스에서는 속성(attribute)라는 표현도 자주 사용한다.)과 데이터 타입(data type)이 명시된 **스키마(schema)** 가 추가 된다는 것이다.

가령, users 라는 데이터셋에 각 사용자의 user_id, name, num_friends에 해당하는 데이터가 있다고 해보자.

In [21]:
users = [[0, "Hero", 0],
        [1, "Dunn", 2],
        [2, "Sue", 3],
        [3, "Chi", 3]]

SQL에서는 이 테이블을 다음과 같이 생성한다.

CREATE TABLE users (

    user_id INT NOT NULL,
    
    name VARCHAR(200),
    
    num_friends INT);

여기서 user_id, num_friends는 정수형(integer)이고, name은 200또는 그 이하의 수의 문자들로 구성되어야 한다고 정의했다. 또한 user_id는 NOT NULL, 즉 값이 빠질 수 없다는 것을 명시했다. (Python으로 이는 not None과 유사하다.)

데이터 행은 **INSERT문** 으로 입력할 수 있다.

INSERT INTO users (user_id, name, num_friends) VALUES (0, "Hero", 0);

SQL문은 항상 세미콜론(;)으로 끝난다. 또한 문자열을 입력할 때는 작은 따옴표(')를 쓴다.

NotQuiteABase에서는 비슷한 종류의 스키마를 명기함으로써 Table이 자동으로 생성될 것이다. 행을 입력하기 위해서는 테이블의 열의 동일한 순서로 작성된 리스트를 insert 함수에 전달해주면 된다.

그러면 실질적으로 함수 내부에서는 **각 열의 이름과 값으로 구성된 딕셔너리로** 변환된 상태로 행이 저장된다. 실제 데이터베이스라면 이렇게 저장 공간을 낭비하는 데이터 구조를 사용하지 않겠지만, 이렇게 하면 NotQuiteABase를 쉽게 다를 수 있다는 이점이 있다.

NotQuiteABase의 Table을 거대한 클래스로 구현할 것이며 메서드를 하나씩 구현할 것이다 우헌 임포트와 타입 별칭(type alias)으로 시작하도록 하자.

In [22]:
from typing import Tuple, Sequence, List, Any, Callable, Dict, Iterable
from collections import defaultdict

# 이후에 사용할 타입 별칭
Row = Dict[str, Any]  # 데이터베이스 행
WhereClause = Callable[[Row], bool]   # 단일 행을 위한 술어
HavingClause = Callable[[List[Row]], bool]  # 여러 행에 대한 술어

생성자부터 시작하자. NotQuiteABase 테이블을 만들기 위해 SQL 데이터베이스에서 테이블을 생성하는 것과 같이, 열의 이름의 리스트와 열 타입의 리스트를 넘겨야 한다.

In [76]:
class Table:
    def __init__(self, columns: List[str], types: List[type]) -> None:
        assert len(columns) == len(types), "# of columns must == # of types"
        
        self.columns = columns  # 열의 이름
        self.types = types   # 열의 타입
        self.rows: List[Row] = []  # (아직 데이터는 없다.)
            
    # 열의 타입을 받기 위한 도우미 메서드 추가
    def col2type(self, col: str) -> type:
        idx = self.columns.index(col)  # 열의 인덱스를 찾고
        return self.types[idx]  # 해당 타입을 반환
    
    # 추가되는 값이 올바른지 확인해 주는 insert 메서드를 추가해 주자.
    # 특히, 맞는 개수의 값을 제공해야 하며 각각이 올바른 타입이어야(혹은 None)한다.
    
    def insert(self, values: list) -> None:
        # 값이 올바른 개수인지 확인
        if len(values) != len(self.types):
            raise ValueError(f"You need to provide {len(self.types)} vlaues")
                
        # 값의 타입이 맞는지 확인
        for value, typ3 in zip(values, self.types):
            if not isinstance(value, typ3) and value is not None:
                raise TypeError(f"Expected type {typ3} but got {value}")
            
        # 대응되는 딕셔너리를 하나의 행으로 추가
        self.rows.append(dict(zip(self.columns, values)))
        
    """
    실제 SQL 데이터베이스에서 특정 열이 빈(null) 값(None)을 가질 수 있는지 여부를 직접 정할 것이다.
    편의상 모든 열이 그럴 수 있다고 가정한다.
    몇 개의 dunder 메서드를 추가, 테이블이 List[Row]처럼 동작할 수 있도록 하여 테스트를 수월하게 만들 것이다.
    """
    
    def __getitem__(self, idx: int) -> Row:
        return self.rows[idx]
    
    def __iter__(self) -> Iterable[Row]:
        return iter(self.rows)
    
    def __len__(self) -> int:
        return len(self.rows)
    
    # 테이블을 예쁘게 출력할 수 있도록 메서드를 추가하자.
    def __repr__(self):
        """테이블을 예쁘게 표현하자: 열 이후에 행"""
        rows = "\n".join(str(row) for row in self.rows)
        
        return f"{self.columns}\n{rows}"
    
    def update(self,
              updates: Dict[str, Any],
              preicate: WhereClause = lambda row: True):
        # 일단 updates가 올바른 이름과 타입을 가졌는지 확인
        for column, new_value in updates.items():
            if column not in self.columns:
                raise ValueError(f"invalid column: {column}")
                
            typ3 = self.col2type(column)
            if not isinstance(new_value, typ3) and new_value is not None:
                raise TypeError(f"expected type {typ3}, but got {new_value}")
        
        for row in self.rows:
            if preicate(row):
                for column, new_value in updates.items():
                    row[column] = new_value
                    
    
    def delete(self, predicate: WhereClause = lambda row: True) -> None:
        """predicate(술부)에 해당하는 모든 행을 제거"""
        self.rows = [row for row in self.rows if not predicate(row)]
        
        
    def select(self,
              keep_columns: List[str] = None,
              additional_columns: Dict[str, Callable] = None) -> 'Table':
        
        if keep_columns is None:  # 만약 특정 열이 명시되지 않았다면
            keep_columns = self.columns  # 모든 행을 반환
            
        if additional_columns is None:
            additional_columns = {}
            
        # 새 열 이름과 타입
        new_columns = keep_columns + list(additional_columns.keys())
        keep_types = [self.col2type(col) for col in keep_columns]
        
        # 타입 어노테이션으로부터 반환 타입을 알아내는 방법은 다음과 같다.
        # 만약 calculation에 반환 타입이 없다면 코드가 멈춰버릴 것이다.
        add_types = [calculation.__annotations__['return']
                    for calculation in additional_columns.values()]
        
        # 결과를 저장하기 위한 새로운 테이블
        new_table = Table(new_columns, keep_types + add_types)
        
        for row in self.rows:
            new_row = [row[column] for column in keep_columns]
            for column_name, calculation in additional_columns.items():
                new_row.append(calculation(row))
            new_table.insert(new_row)
            
        return new_table
    
    def where(self, preicate: WhereClause = lambda row: True) -> 'Table':
        """주어진 predicate에 해당하는 행만 반환"""
        where_table = Table(self.columns, self.types)
        for row in self.rows:
            if preicate(row):
                values = [row[column] for column in self.columns]
                where_table.insert(values)
        return where_table
    
    def limit(self, num_rows: int) -> 'Table':
        """첫 num_rows 만큼의 행만 반환"""
        limit_table = Table(self.columns, self.types)
        for i, row in enumerate(self.rows):
            if i >= num_rows:
                break
            values = [row[column] for column in self.columns]
            limit_table.insert(values)          
        return limit_table
    
    def group_by(self,
             group_by_columns: List[str],
             aggregates: Dict[str, Callable],
             having: HavingClause = lambda group: True) -> 'Table':

        grouped_rows = defaultdict(list)

        # 그룹을 채워 넣는다.
        for row in self.rows:
            key = tuple(row[column] for column in group_by_columns)
            grouped_rows[key].append(row)

        # 결과 테이블은 group_by 열과 병합된 열들로 구성된다.
        new_columns = group_by_columns + list(aggregates.keys())
        group_by_types = [self.col2type(col) for col in group_by_columns]
        aggregate_types = [agg.__annotations__['return']
                           for agg in aggregates.values()]
        result_table = Table(new_columns, group_by_types + aggregate_types)

        for key, rows in grouped_rows.items():
            if having(rows):
                new_row = list(key)
                for aggregate_name, aggregate_fn in aggregates.items():
                    new_row.append(aggregate_fn(rows))
                result_table.insert(new_row)

        return result_table

    def order_by(self, order: Callable[[Row], Any]) -> 'Table':
        new_table = self.select()       # make a copy
        new_table.rows.sort(key=order)
        return new_table
    

    def join(self, other_table: 'Table', left_join: bool = False) -> 'Table':

        join_on_columns = [c for c in self.columns           # columns in
                           if c in other_table.columns]      # both tables

        additional_columns = [c for c in other_table.columns # columns only
                              if c not in join_on_columns]   # in right table

        # all columns from left table + additional_columns from right table
        new_columns = self.columns + additional_columns
        new_types = self.types + [other_table.col2type(col)
                                  for col in additional_columns]

        join_table = Table(new_columns, new_types)

        for row in self.rows:
            def is_join(other_row):
                return all(other_row[c] == row[c] for c in join_on_columns)

            other_rows = other_table.where(is_join).rows

            # Each other row that matches this one produces a result row.
            for other_row in other_rows:
                join_table.insert([row[c] for c in self.columns] +
                                  [other_row[c] for c in additional_columns])

            # If no rows match and it's a left join, output with Nones.
            if left_join and not other_rows:
                join_table.insert([row[c] for c in self.columns] +
                                  [None for c in additional_columns])

        return join_table

User 테이블을 만들자

In [60]:
# 생성자는 열의 이름과 타입을 필요로 한다.

users = Table(['user_id', 'name', 'num_friends'], [int, str, int])
users.insert([0, "Hero", 0])
users.insert([1, "Dunn", 2])
users.insert([2, "Sue", 3])
users.insert([3, "Chi", 3])
users.insert([4, "Thor", 3])
users.insert([5, "Clive", 2])
users.insert([6, "Hicks", 3])
users.insert([7, "Devin", 2])
users.insert([8, "Kate", 2])
users.insert([9, "Klein", 3])
users.insert([10, "Jen", 1])

이제 print(users)를 실행하면 아래와 같은 출력을 볼 수 있다.

In [61]:
print(users)

['user_id', 'name', 'num_friends']
{'user_id': 0, 'name': 'Hero', 'num_friends': 0}
{'user_id': 1, 'name': 'Dunn', 'num_friends': 2}
{'user_id': 2, 'name': 'Sue', 'num_friends': 3}
{'user_id': 3, 'name': 'Chi', 'num_friends': 3}
{'user_id': 4, 'name': 'Thor', 'num_friends': 3}
{'user_id': 5, 'name': 'Clive', 'num_friends': 2}
{'user_id': 6, 'name': 'Hicks', 'num_friends': 3}
{'user_id': 7, 'name': 'Devin', 'num_friends': 2}
{'user_id': 8, 'name': 'Kate', 'num_friends': 2}
{'user_id': 9, 'name': 'Klein', 'num_friends': 3}
{'user_id': 10, 'name': 'Jen', 'num_friends': 1}


In [62]:
assert len(users) == 11
assert users[1]["name"] == "Dunn"

## 24.2 UPDATE

데이터를 입력한 후 수정, 또는 업데이트해야 할 때가 있다. e.g. Dunn에게 친구가 한 명 더 생겼다고 하면, SQL로 다음과 같은 쿼리를 던질 것이다.

UPDATE users

SET num_friends = 3

Where user_id = 1;

이때 명시해야 하는 것을 아래와 같다.

- 어떤 테이블을 업데이트할 것인가

- 어떤 행을 업데이트할 것인가

- 어떤 필드를 업데이트할 것인가

- 어떤 값으로 치환할 것인가

NotQuiteABase에도 비슷하게 update라는 함수를 추가해보자. 함수의 첫 번째 인자에는 키가 업데이트할 열, 값이 치환하고자는 하는 필드의 새로운 값인 딕셔너리를 전달할 것이다. (선택적인)두 번째 인자에는 업데이트하고 싶은 행에 대해서만 True를 반환하고 그렇지 않은 경우에는 False를 반환해 주는 predicate라는 함수를 전달할 것이다. 

그리고 다음과 같이 이용할 수 있다.

In [63]:
assert users[1]['num_friends'] == 2

In [64]:
users.update({'num_friends':3},  # num_friends = 3으로 설정
            lambda row: row['user_id'] == 1)  # user_id == 1인 행에 대해

In [65]:
assert users[1]['num_friends'] == 3

## 24.3 DELETE

SQL로 테이블의 행을 지우는 방법은 두 가지이다. 먼저, 테이블의 모든 행을 지우는 위험한 방법이 있다.

DELETE FROM users;

덜 위험한 방법으로 특정 조건(condition)을 만족하는 행만 **WHERE절** 을 추가하여 제거하는 방법이 있다.

DELETE FROM users WHERE user_id = 1;

In [66]:
def delete(self, predicate: WhereClause = lambda row: True) -> None:
        """predicate(술부)에 해당하는 모든 행을 제거"""
        self.rows = [row for row in self.rows if not predicate(row)]

WHERE절과 같은 predicate(술부) 함수를 전달하면 그 조건에 맞는 행만 삭제할 것이다. predicate를 전달하지 않으면 기본적으로 항상 True를 반환하기 때문에 모든 행을 삭제할 것이다. 

users.delete(lambda row: row["user_id"] == 1)   # user_id == 1인 행을 삭제

users.delete()  # 모든 행을 삭제

## 24.4 SELECT
SQL 테이블을 직접 들여다 볼 일은 그렇게 많지 않을 것이다. 대신 SELECT문을 이용해서 필요한 몇 가지 행만 선택해서 보는 경우가 많다.

SELECT * FROM users;  -- 모든 내용을 선택

SELECT * FROM users LIMIT 2;  -- 첫 두 행만 선택

SELECT user_id FROM users;  -- 특정 열만 선택

SELECT user_id FROM users WHERE name == 'Dunn';  -- 특정 행만 선택

SELECT 문은 필드값을 계산하기 위해 사용하기도 한다.

SELECT LENGTH(name) AS name_length FROM users;


이제 Table에 select 메서드를 추가해 보자. 이 함수는 새로운 Table을 반환할 것이고, 두 개의 선택적 인자를 전달받는다.

- keep_columns엔 보존하고 싶은 열의 이름을 입력할 수 있다. 명시하지 않으면 기본적으로 모든 열을 보존한다.

- additional_columns에는 새로 추가하고 싶은 열을 딕셔너리의 형태로 입력할 수 있다. 이대 딕셔너리의 키는 추가하는 열의 이름이 되고, 값은 새 열의 값을 계산하는 방법을 명시한 함수이다. 해당 함수의 타입 어노테이션을 살펴보고 이로부터 새 열의 타입을 알아낼 것이다. 그러므로 함수가 반환 타입을 어노테이션해 줘야 한다.

두 인자 중 아무것도 전달하지 않으면 입력한 테이블과 같은 테이블이 그대로 반환된다.

한 가지 유의할 점은, select 메서드는 아예 새로운 Table을 반환하는데 반해 실제 SQL의 SELECT문은 결괏값을 테이블에 넣어 달라고 명시하지 않으면 화면에 출력되기만 한다.

여기서 where 그리고 limit 메서드까지 추가해 보자. 

In [67]:
# SELECT * FROM users;
all_users = users.select()
assert len(all_users) == 11

In [68]:
# SELECT * FROM users LIMIT 2;
two_users = users.limit(2)
assert len(two_users) == 2

In [69]:
# SELECT user_id FROM users;
just_ids = users.select(keep_columns=["user_id"])
assert just_ids.columns == ['user_id']

In [70]:
# SELECT user_id FROM users WHERE name = 'Dunn';
dunn_ids = (users.where(lambda row: row["name"] == "Dunn").select(keep_columns=["user_id"]))
assert len(dunn_ids) == 1
assert dunn_ids[0] == {"user_id":1}

In [71]:
# SELECT LENGTH(name) AS name_length FROM users;
def name_length(row) -> int: return len(row["name"])

In [72]:
new_lengths = users.select(keep_columns=[],
                         additional_columns={"name_length": name_length})

assert name_lengths[0]['name_length'] == len("Hero")

NameError: name 'name_lengths' is not defined

## 24.5 GROUP BY

빈번하게 쓰이는 또 다른 SQL문 중 하나는 GROUP BY이다. GROUP BY는 지정된 열에서 동일한 값을 가진 행을 묶어 주며 MIN이나 MAX, COUNT, SUM 등의 병합 계산을 할 수 있게 한다.

예를 들어 이름의 길이에 따른 상용자의 수와 가장 작은 user_id를 알고 싶을 수 있다.

SELECT LENGTH(name) as name_length, MIN(user_id) AS min_user_id, COUNT(*) AS num_users

FROM users

GROUP BY LNEGTH(name);

SELECT를 통해 선택된 모든 데이터 필드는 name_length처럼 GROUP BY절에 포함되어 있거나 min_user_id, num_users 처럼 병합 계산되어야 한다.

SQL에는 WHERE절과 매우 유사하게 작동하는 HAVING절도 있는데, 병합을 하기 전에 행을 필터링하는 WHERE절과 달리 HAVING절은 병합된 결과를 필터링한다.

또는 이름이 특정 글자로 시작하는 사용자들의 평균 친구 수를 보고 싶을 수도 있다. 특히 그중에서도 친구 수가 평균적으로 1 이상인 경우만 출력해 보자.

SELECT SUBSTR(name, 1, 1) AS first_lette,

AVG(num_friends) AS avg_num_friends

FROM users

GROUP BY SUBSTR(name, 1, 1)

HAVING AVG(num_friends) > 1;

NotQuiteABase Table에 이 기능을 추가해보자. 아래의 단계를 거쳐서 작동한다.

1. (그룹 지어진 값의) tuple을 (그룹된 값을 포함하는) 행으로 변환하는 defaultdict를 만든다. 리스트는 딕셔너리의 키로 사용할 수 없고, 대신 tuple을 사용해야 한다는 점을 상기하자.

2. 테이블의 행을 훑으며 defaultdict를 채운다.

3. 올바른 출력 열을 포함하는 새로운 테이블을 생성한다.

4. defaultdict를 순회하며 having 필터를 적용하고, 새로운 테이블에 값을 채워넣는다.

## 24.6 ORDER BY

결괏값을 정렬하고 싶을 때는 어떻게 하면 될까? 예를 들어 사용자의 이름을 알파벳 순으로 정렬한 후 앞의 이름 두 개만 얻고 싶다면 어떻게 해야 할까?

SELECT * FROM users

ORDER BY name

LIMIT 2;

이 SQL문을 우리의 Table에 구현하기 위해 order라는 함수를 입력값으로 받는 order_by라는 새로운 메서드르 만들어 보자.

## 24.7 JOIN

관계형 데이터베이스는 종종 중복을 최소화하도록 정규화(Normalize)되어 있다. 예를 들어 파이썬으로 사용자 관심사를 다룰 때는 단순히 각 사용자에 대해 관심사를 리스트로 표현하면 됐었다.

SQL 테이블을 보통 리스트를 담고 있지 않으므로 보통의 해결책은 새로운 테이블인 user_interests 를 생성해서 user_id와 관심사 interest사이에 1:N 간계를 성립하게 하는 것이다. 이를 SQL로 표현하면 아래와 같다.

CREATE TABLE user_interests (
    user_id INT NOT NULL,
    interest VARCHAR(100) NOT NULL);

NotQuiteABase에서는 다음과 같이 테이블을 생성할 수 있다.

In [75]:
user_interests = Table(['user_id', 'interest'], [int, str])
user_interests.insert([0, "SQL"])
user_interests.insert([0, "NoSQL"])
user_interests.insert([2, "SQL"])
user_interests.insert([2, "MySQL"])

데이터가 서로 다른 테이블에 있을 때, 어떻게 분석해야 할까? 이럴 때 샤용하는게 바로 JOIN이다. JOIN은 왼쪽에 있는 테이블의 행을 오른쪽에 있는 테이블의 행과 매칭(matching)시켜 준다. 이때 우리는 어떻게 '매칭'시킬 것인지만 정의해주면 된다.

## 24.8 서브 쿼리

SQL에서는 쿼리를 결과물을 마치 또 다른 테이블인 것처럼 간주하고 SELECT(또는 JOIN)를 할 수 있다. 예를 들어 SQL에 관심 있는 사용자들 중에서 user_id가 가장 작은 사용자를 반환하고 싶을 때 이런 서브쿼리(subquery)를 쓰면 좋다. 

SELECT MIN(user_id) AS min_user_id FROM (SELECT use_id FORM user_interests WHERE interest = 'SQL') sql_interests;

## 24.9 인덱스

특정 값을 가진 행을 찾을 때, NotQuiteABase는 테이블의 모든 행을 훑어야 한다. 그리고 행의 수가 아주 많다면 시간도 무척 오래 걸릴 것이다.

직접 만든 join 알고리즘 역시 무척 비효율적인다. 왼쪽 테이블의 각 행별로, 오른쪽 테이블의 모든 행을 한 번씩 훑으며 매칭되는 행이 있는지 찾기 때문이다. 두 테이블 모두 크다면, 이 과정은 엄청나게 오래 걸릴 수 있다.

게다가 때로는 열에 제약 조건을 주고 싶을 때도 있다. 예를 들어 users 테이블에서 두 명 이상의 사람들이 같은 user_id를 갖지 못하게 하고 싶을 수 있다.

인덱스(index)가 이 모든 문제를 해결해 준다. user_interests 테이블에 user_id에 대한 인덱스가 있다면 join 알고리즘은 전체 테이블을 훑어보지 않고도 매칭되는 행을 찾을 수 있게 된다. 또한 user_id 인덱스에 유일성(unique) 조건이 걸려있다면, 중복된 값을 생성하려고 할 때 에러가 발생하도록 만들 수 있다.

데이터 베이스의 각 테이블에는 하나 또는 그 이상의 인덱스가 있을 수 있고, 이 덕분에 중요한 열의 행을 빠르게 찾을 수 있으며, 효울적으로 테이블을 JOIN하거나 열 또는 열의 조합에 유일성 제약 조건을 추가할 수 있다.

한편 인덱스를 ㅁ나들고 사용하는 것은 일종의 흑마법과 같다. 하지만 데이터베이스를 자주 다룰 거라면 제대로 배워 볼 가치가 있다.

## 24.10 쿼리 최적화

SQL에 관심 있는 사용자를 찾는 쿼리문은 다음과 같다.

SELECT user.name \
FROM users \
JOIN user_interests \
ON users.user_id = user_interests.user_id \
WHERE user_interests.interest = 'SQL'

SQL에서는 원하는 결과를 '선언'하고 나면, 쿼리 엔진이 알아서 어떤 방식으로 실행될 것인지(그리고 어떤 인덱스를 사용할 것인지) 결정하기 때문이다.

## 24.11 NoSQL

데이터베이스의 최근 트렌드 중 하나는 데이터를 테이블로 표현하지 않는 비관계형(Nonrelational) 데이터베이스, 즉 NoSQL이다. 에를 들어 NongoDB는 스키마와 행을 사용하지 않고 JSON 문서의 형태로 데이터를 표현하는 데이터베이스이다.

그 외에도 행 대신 열에 데이터를 저장하는 컬럼형 데이터베이스도 있고 (이런 데이터 베이스는 열의 수가 아주 많지만, 그 중 일부만 쿼리에 사용할 때 좋다.)

키로 단일 혹은 복잡한 값을 반환하는 키-값 스토리지, 그래프를 저장하고 순회하기 위한 데이터베이스, 여러 데이터 센터 간 호환성을 고려한 데이터베이스, 메모리상에서 동작하는 데이터베이스, 시계열 데이터를 저장하는 데이터베이스 등등 다양한 형태의 데이터베이스가 있다.